# Stock & Convertible Bond Arbitrage 0001

## 1. Import Library

In [62]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [63]:
sd = '2024-12-31'
ed = '2026-02-13'

In [64]:
sql = """
SELECT
    date,
    instrument,
    stock_code,
    1 * c_normalize(transfer_value) - 1 * c_normalize(conversion_premium_rate) + 0.5 * c_normalize(implied_volatility) - 0.5 * c_normalize(pure_debt_value) AS score,
    IF(score < 1.0 AND m_lag(score, 1) > 1.0, 1, 0) AS signal_stock,
    IF(score > 1.0 AND m_lag(score, 1) < 1.0, 1, 0) AS signal_cbond,
FROM cn_cbond_analyze_metric JOIN cn_cbond_basic_info USING (instrument)
QUALIFY (signal_stock = 1 OR signal_cbond = 1)
ORDER BY date
"""

In [65]:
df_strategy = dai.query(sql, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,stock_code,score,signal_stock,signal_cbond
0,2025-01-02,127076.SZ,002891.SZ,1.117242,0,1
1,2025-01-02,123182.SZ,300900.SZ,0.904584,1,0
2,2025-01-02,110060.SH,600326.SH,1.032463,0,1
3,2025-01-02,110097.SH,600419.SH,1.158153,0,1
4,2025-01-02,123025.SZ,300567.SZ,0.243181,1,0
...,...,...,...,...,...,...
1689,2026-02-13,123224.SZ,301266.SZ,0.995766,1,0
1690,2026-02-13,123160.SZ,300992.SZ,1.798585,0,1
1691,2026-02-13,113677.SH,603306.SH,1.142393,0,1
1692,2026-02-13,128137.SZ,002859.SZ,0.149559,1,0


## 3. Backtest Strategy

In [66]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    instruments_hold = set(context.get_account_positions().keys())

    for i, x in df_now.iterrows():
        
        if x.signal_stock == 1:
            if x.instrument in instruments_hold:
                context.order(x.instrument, 0)
            context.order(x.stock_code, 100)
        
        if x.signal_cbond == 1:
            if x.stock_code in instruments_hold:
                context.order(x.stock_code, 0)
            context.order(x.instrument, 100)
    
    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 21:52:43] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 21:52:43] [info     ] 2025-01-02, 2026-02-13, , , instruments=235
[2026-02-18 21:52:43] [info     ] bigtrader module V2.2.0
[2026-02-18 21:52:43] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
